<a href="https://colab.research.google.com/github/febuz/jhehTAlib/blob/main/jhTAlib_returngazer_computing_environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is focussed on a feather light RAM usage with ensemble learning (libxgboost-1.3.3dev., rapids-xgboost-0.18., xgboost1.3.3dev.rap) and 21-30 million rows processing power

# This notebook is focussed on using jhTAlib for Data Scientists participating in the Datacrunch competition where only the US stockmarket is being used

# Sample OpenSource dataset is compiled with care from public sources like Yahoo, FMP and BvD; IDs are gained from the asset managers who participate in Datacrunch

# Authors are JH, Joost Hoeks and Edwin Hauwert, EH


```
# Code credits go to JH
# Data credits go to EH
Free to change every aspect of this notebook, copy it etc. 
as long as you keep the jhTAlib library
as well as the dataset 
that comes with this ipynb
```



https://github.com/joosthoeks/jhTAlib/stargazers 

In [ ]:
# Hardware check RAM, CPU
!free -h
!cat /proc/cpuinfo

              total        used        free      shared  buff/cache   available
Mem:            12G        565M         10G        1.0M        2.0G         11G
Swap:            0B          0B          0B
processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu

In [1]:
# for timing your code duration per cell
import timeit

In [2]:
# install and import pyspark Spark for Python
!pip install pyspark
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession

# Initialize pyspark
sc = pyspark.SparkContext('local[*]')

     |████████████████████████████████| 212.3MB 74kB/s 
     |████████████████████████████████| 204kB 41.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=293fa94486858e165999a8c7101dfb5e294268f934ba467af49f18ae1e1b4de7
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [3]:
%%time
# Get packages
!pip install --upgrade jhtalib

""""""
# Import Built-Ins:
from pprint import pprint as pp
import sys, os
import numpy as np

# Import Third-Party:
import pynvml
import pyarrow as pa

# Import Homebrew:
import jhtalib as jhta


# df is DataFeed:
df = {
    'datetime': ('20151217', '20151218', '20151221', '20151222', '20151223', '20151224', '20151228', '20151229', '20151230', '20151231'),
    'Open': (235.8, 232.3, 234.1, 232.2, 232.7, 235.4, 236.9, 234.85, 236.45, 235.0),
    'High': (238.05, 236.9, 237.3, 232.4, 235.2, 236.15, 236.9, 237.6, 238.3, 237.25),
    'Low': (234.55, 230.6, 230.2, 226.8, 231.5, 233.85, 233.05, 234.6, 234.55, 234.4),
    'Close': (234.6, 233.6, 230.2, 230.05, 234.15, 236.15, 233.25, 237.6, 235.75, 234.4),
    'Volume': (448294, 629039, 292528, 214170, 215545, 23548, 97574, 192908, 176839, 69347)
     }

# basic usage:
#pp (df)
pp (jhta.SMA(df, 10)[9])
#pp (jhta.BBANDSgr(df, 10))

     |████████████████████████████████| 51kB 6.7MB/s 
233.97500000000005
CPU times: user 51.8 ms, sys: 21.6 ms, total: 73.4 ms
Wall time: 2.78 s


In [4]:
%%time
# Test Pyspark
words = "to be or not to be".split()
words
words_rdd = sc.parallelize(words)
words_rdd

nSlices = 2

def mult(x):
    y = np.array([2])
    return x*y

sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10],4).map(lambda x: x**2).sum()

x = np.arange(10000)
distData = sc.parallelize(x,nSlices)
 
results = distData.map(mult).collect()
 
print(results)

[array([0]), array([2]), array([4]), array([6]), array([8]), array([10]), array([12]), array([14]), array([16]), array([18]), array([20]), array([22]), array([24]), array([26]), array([28]), array([30]), array([32]), array([34]), array([36]), array([38]), array([40]), array([42]), array([44]), array([46]), array([48]), array([50]), array([52]), array([54]), array([56]), array([58]), array([60]), array([62]), array([64]), array([66]), array([68]), array([70]), array([72]), array([74]), array([76]), array([78]), array([80]), array([82]), array([84]), array([86]), array([88]), array([90]), array([92]), array([94]), array([96]), array([98]), array([100]), array([102]), array([104]), array([106]), array([108]), array([110]), array([112]), array([114]), array([116]), array([118]), array([120]), array([122]), array([124]), array([126]), array([128]), array([130]), array([132]), array([134]), array([136]), array([138]), array([140]), array([142]), array([144]), array([146]), array([148]), arra

In [7]:
%%time
# Takes about 15 min te set up the environment for efficient computing with parquet files
# RAPIDS & CO: https://medium.com/dropout-analytics/installing-rapids-ai-in-google-colab-87c247f2c468
!which python # should return /usr/local/bin/python
!python --version
!echo $PYTHONPATH
# unset Pythonpath
%env PYTHONPATH=
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)
pp (device_name)
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable
dist_package_index = sys.path.index('/usr/local/lib/python3.7/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.7/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

# How to make this SC with parallize function work in colab after installing RAPIDS?
words_rdd = sc.parallelize(words)
words_rdd

# How to use cudatoolkit-11.0.221, libcudf-0.18.1, dask-2021.4.0, dask-cuda-0.18.0, libcugraph-0.18.0 and cudnn-8.0.0?

# from sklearn.linear_model import LogisticRegression
from cuml.linear_model import LogisticRegression

# for use of arrow-cpp-proc-3.0.0, parquet-ccp, and parquet-cpp-1.5.1 see further



/usr/local/bin/python
Python 3.7.10
/env/python
env: PYTHONPATH=
b'Tesla T4'
Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 213 (delta 22), reused 3 (delta 0), pack-reused 171
Receiving objects: 100% (213/213), 64.29 KiB | 12.86 MiB/s, done.
Resolving deltas: 100% (84/84), done.
PLEASE READ
********************************************************************************************************
Changes:
1. IMPORTANT SCRIPT CHANGES: Colab has updated to Python 3.7, and now runs our STABLE and NIGHTLY versions (0.18 and 0.19)!  PLEASE update your older install script code as follows:
	!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.18

	import sys, os

	dist_package_index = sys.path.index('/usr/local/lib/python3.7/dist-packages')
	sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.7/site-packages'] + sys.path[dist_package_index:]
	sys.

In [8]:
%%time
import pyarrow as pa
# https://stackoverflow.com/questions/57683276/how-to-convert-numpy-to-parquet-without-using-pandas/65464002#65464002

df = jhta.CSVURL2DF('https://raw.githubusercontent.com/joosthoeks/jhTAlib/master/example/data.csv')
#pp (df['Close'])

df_np = {
    'datetime': np.array(df['datetime']),
    'Open': np.array(df['Open'], dtype=float),
    'High': np.array(df['High'], dtype=float),
    'Low': np.array(df['Low'], dtype=float),
    'Close': np.array(df['Close'], dtype=float),
    'Volume': np.array(df['Volume'], dtype=int)
}
#pp (df_np['Close'])

pa_table = pa.table({
    'datetime': df_np['datetime'],
    'Open': df_np['Open'],
    'High': df_np['High'],
    'Low': df_np['Low'],
    'Close': df_np['Close'],
    'Volume': df_np['Volume']
})
pp (pa_table['Close'])

pa.parquet.write_table(pa_table, 'test.parquet')


[
  [
    182.15,
    184.4,
    184.45,
    185.7,
    184.45,
    184.7,
    186.25,
    187.1,
    186.15,
    186.8,
    ...
    234.6,
    233.6,
    230.2,
    230.05,
    234.15,
    236.15,
    233.25,
    237.6,
    235.75,
    234.4
  ]
]
CPU times: user 8.42 ms, sys: 3.12 ms, total: 11.5 ms
Wall time: 142 ms


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# bashrc
#os.environ['PATH'] = os.environ['JAVA_HOME'] + "/bin:" + os.environ['PATH']
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Set your spark folder to your system path environment.
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"
#os.environ['SPARK_HOME'] = "/content//spark-3.0.0-bin-hadoop3.2"
#os.environ['PYTHONPATH'] = "$SPARK_HOME/python:$SPARK_HOME/python/lib/py4j-0.10.4-src.zip:$PYTHONPATH"
#os.environ['PATH']="$SPARK_HOME/bin:$SPARK_HOME/python:$PATH"

# checks if Spark works


In [ ]:
%%time
# Install findspark using pip
!pip install -q findspark

# setup SPARK and pyspark To use python with Apache SPARK, spark nlp to get spark config
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash
!pip install spark-nlp==3.0.1 pyspark==3.1.1
import findspark
findspark.init()

In [ ]:
%%time
# Test Pyspark 2, is time shorter than in the first test? why does it start disfunctioning after the install
words = "to be or not to be".split()
words
words_rdd = sc.parallelize(words)
words_rdd

nSlices = 2

def mult(x):
    y = np.array([2])
    return x*y

sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10],4).map(lambda x: x**2).sum()

x = np.arange(10000)
distData = sc.parallelize(x,nSlices)
 
results = distData.map(mult).collect()
 
print(results)

In [ ]:
%%time
import io
import pandas as pd
from google.colab import files
#import json from financialmodelingprep
import json
uploaded = files.upload()
df = pd.read_json(io.StringIO(uploaded.get('Missing_shares_BZ.json').decode('utf-8')))
df.to_csv('json_unpack_sharesBZ.csv')
print(df)
#import cudf
#data = cudf.read_csv("../content/train.csv")
#print("Train size:", data.shape)

Saving Missing_shares_BZ.json to Missing_shares_BZ (1).json
    symbol  ...           timestamp
0      ABM  ... 2021-04-13 21:39:05
1     ALXO  ... 2021-04-13 21:39:05
2      BIG  ... 2021-04-13 21:39:05
3       BJ  ... 2021-04-13 21:39:05
4       BL  ... 2021-04-13 21:39:05
..     ...  ...                 ...
321    WWW  ... 2021-04-13 21:39:05
322      X  ... 2021-04-13 21:39:05
323   ZIXI  ... 2021-04-13 21:39:05
324   ZNTL  ... 2021-04-13 21:39:05
325    ZUO  ... 2021-04-13 21:39:05

[326 rows x 22 columns]
CPU times: user 136 ms, sys: 17 ms, total: 153 ms
Wall time: 9.25 s


# Get US Shares data

In [ ]:
!pip install numerapi==2.3.8
!pip install yfinance
#!pip install git+https://github.com/leonhma/yfinance.git #drop-in replacement yfinance fork for failed downloads, h/t ceunen
!pip install simplejson
import numerapi
import yfinance
import simplejson
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests as re
from datetime import datetime
from dateutil.relativedelta import relativedelta, FR
import requests
from bs4 import BeautifulSoup
NAPI_PUBLIC_KEY = 'ENTER_PUBLIC_KEY'
NAPI_PRIVATE_KEY = 'ENTER_PRIVATE_KEY'
napi = numerapi.SignalsAPI(NAPI_PUBLIC_KEY, NAPI_PRIVATE_KEY)
# read in list of active Signals tickers which can change slightly era to era
eligible_tickers = pd.Series(napi.ticker_universe(), name='ticker') 
print(f"Number of eligible tickers: {len(eligible_tickers)}")
# read in yahoo to numerai ticker map, still a work in progress, h/t wsouza and 
# this tickermap is a work in progress and not guaranteed to be 100% correct
ticker_map = pd.read_csv('https://numerai-signals-public-data.s3-us-west-2.amazonaws.com/signals_ticker_map_w_bbg.csv')
ticker_map = ticker_map[ticker_map.bloomberg_ticker.isin(eligible_tickers)]

numerai_tickers = ticker_map['bloomberg_ticker']
yfinance_tickers = ticker_map['yahoo']
print(f"Number of eligible tickers in map: {len(ticker_map)}")
raw_data = yfinance.download(yfinance_tickers.str.cat(sep=' ')[], start='2002-12-01', threads=True) 
print(raw_data.columns)
full_data = raw_data['Adj Close'].stack().reset_index()

#takes >30 min, too long without result check manual first
#def get_n_shares_outstanding(ticker):
#    nasdaq_soup = requests.get("https://www.nasdaq.com/symbol/{}/stock-report".format(ticker))
#    soup = BeautifulSoup(nasdaq_soup.content, "html.parser")#

#    for td in soup.find_all("th",text="Shares Outstanding"):
#        n_shares_outstanding = int(td.find_next_sibling().get_text().replace(',', ''))##
#    return n_shares_outstanding


#get_n_shares_outstanding('AAPL')

In [ ]:
# Quandl python api retreive shares example
!pip install quandl
import quandl
start_date = '2020-07-06'
collapse = 'monthly'
authtoken = 'QNKZ62RyQQxU3B3jSpTX'
authtoken2 = '5WBJ1BRRCBY0KTKD'
quandl.ApiConfig.api_key = authtoken
key = os.environ.get(authtoken2)
print(key)

# Sample ZACKS data
#mydata3 = quandl.get_table('ZACKS/FC', ticker='MSFT')
print(mydata3.wavg_shares_out)
#mydata4 = quandl.get_table('ZACKS/SHRS', ticker='MSFT')
print(mydata4.avg_d_shares)
#mydata5 = quandl.get_table('ZACKS/SHRS', ticker='MSFT')
print(mydata5.shares_out)

# Sample data WVB Bureau van Dijk OSIRIS data
mydata6 = quandl.get_table('WVB/WC')
#https://wvb.com/corporations/list/index/company/aapl/country/all/sic/all/fta/all/alphabet//corporate_filter/Search/
print(mydata6[0:1])
#mydata7 = quandl.get_table('WVB/SO', company_perm_id='MSFT')
#print(mydata7.total_shares_outstanding)
